In [ ]:
!pip install -r requirements.txt

In [1]:
import ipywidgets as widgets
from IPython.display import clear_output
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pycountry
import pycountry_convert as pc
from typing import Tuple

from google.colab import output
output.enable_custom_widget_manager()

In [2]:

class BIP_data():
  def __init__(self, path: str) -> None:
    super().__init__()
    self.path = path
    self.bip_df, self.bip_pc_df = self._preprocess()
    self.countries = None

  def _preprocess(self) -> Tuple[pd.DataFrame]:
    data = pd.read_table(self.path)
    data[["na_item","unit","geo"]] = data.iloc[:,0].str.split(",", expand=True)
    data.drop(data.columns[0], axis=1, inplace=True)
    data = data[data.columns[-3:].tolist() + data.columns[:-3].tolist()]
    data.drop("na_item", axis=1, inplace=True)

    data.iloc[:,2:] = data.iloc[:,2:].replace("[a-zA-Z: ]", "", regex=True).replace("", 0, regex=True).astype(np.float32)

    # fix country codes
    data.drop(data[data.geo.str.len() > 2].index, inplace=True)
    self._format_country_codes(data)

    data["geo"] = data["geo"].apply(self.alpha2_to_alpha3)

    bip_types = data["unit"].unique()

    bip_pc_data, bip_data = map(lambda x: data[data["unit"] == x], bip_types)

    for df in [bip_data, bip_pc_data]:
      df.drop(columns=["unit"], axis=1, inplace=True)

    return bip_data.reset_index(drop=True, inplace=True), bip_pc_data

  def _format_country_codes(self, df: pd.DataFrame) -> None:

    invalid_codes = ["UK", "EA", "XK"]

    df.drop(df[df["geo"].isin(invalid_codes)].index, inplace=True)
    df["geo"].replace("EL", "GR", inplace=True)

  def alpha2_to_alpha3(self, country_alpha2: str) -> str:
    country_name = pc.country_alpha2_to_country_name(country_alpha2)
    country_alpha3 = pc.country_name_to_country_alpha3(country_name)
    return country_alpha3


              


In [ ]:
test = BIP_data("https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/tec00001.tsv.gz")

In [4]:
bip = test.bip_df

In [5]:
bip_pc = test.bip_pc_df

In [97]:
fig = go.FigureWidget()

for i, country in enumerate(bip["geo"]):
  visiblity = None

  if country == "DEU":
    visibility = None
  else:
    visibility = "legendonly"

  fig.add_trace(go.Scatter(
      x=bip.columns[1:],
      y=bip.iloc[i,1:],
      name=country,
      visible=visibility
  ))

fig.update_layout(
    title="GDP of european countries",
    xaxis_title="Year",
    yaxis_title="GDP in million euro",
    legend_title="Countries",
)

fig.show()

In [129]:
bip_melted = bip.melt(id_vars="geo", value_vars=bip.columns[2:], var_name="year")
bip_melted["geo"] = bip_melted["geo"].astype(str)
bip_melted["value"] = bip_melted["value"].astype(np.float32)
bip_melted["year"] = bip_melted["year"].astype(int)


In [377]:
fig3 = px.choropleth(
          bip_melted,
          locations='geo', 
          color='value',
          scope="europe",
          title='Total GDP',
          range_color=(bip_melted["value"].min(), bip_melted["value"].max()),
          height=600,
          animation_frame="year",
          basemap_visible=True
        )
fig3.update_geos(resolution=50)
fig3.update_layout(margin={"r":60,"t":60,"l":50,"b":50})
fig3.update_layout(coloraxis={
    "colorbar":{"x":-0.2,
                "title":{"text":"GDP in million EUR", "side":"right"}}})

fig3.show()

In [9]:
unemployment = pd.read_table("https://ec.europa.eu/eurostat/estat-navtree-portlet-prod/BulkDownloadListing?file=data/tipsun20.tsv.gz")

In [10]:
new_cols = ["sex", "age", "unit", "geo"]

unemployment[new_cols] = unemployment.iloc[:,0].str.split(",", expand=True)
unemployment.drop(unemployment.columns[0], axis=1, inplace=True)

In [11]:
unemployment = unemployment[unemployment.columns[-len(new_cols):].tolist() + unemployment.columns[:-len(new_cols)].tolist()]

In [12]:
unemployment.drop([unemployment.columns[0],unemployment.columns[2]], axis=1, inplace=True)

In [13]:
unemployment.iloc[:,2:] = unemployment.iloc[:,2:].replace("[a-zA-Z: ]", "", regex=True).replace("", 0, regex=True).astype(np.float32)

In [14]:
def iso2_to_iso3(iso2):
  country = pycountry.countries.get(alpha_2=iso2)
  # Old Code for Greek is EL -> the new one is GR (iso2) or GRC (iso3)
  if(iso2 == 'EL'):
    return 'GRC'
  return country.alpha_3

In [15]:
unemployment["geo"] = unemployment["geo"].apply(iso2_to_iso3)

In [378]:
country_w = widgets.Dropdown(
    options=unemployment["geo"].unique(),
    value='DEU',
    description='Country:')

age_group_w= widgets.Dropdown(options=unemployment["age"].unique(),
                     value="Y15-24",
                     description="Agegroup:")

fig2 = go.FigureWidget(make_subplots(specs=[[{"secondary_y": True}]]))


unemployment_index = unemployment[(unemployment["geo"] == country_w.value) & (unemployment["age"] == age_group_w.value)].index[0]
fig2.add_trace(go.Scatter(
      x=unemployment.columns[9:],
      y=unemployment.iloc[unemployment_index,9:],
      name=age_group_w.value,
  ),secondary_y=True)

bip_index = bip[bip["geo"] == country_w.value].index[0]
fig2.add_trace(go.Scatter(
      x=bip.columns[1:],
      y=bip.iloc[bip_index,1:],
      name=country_w.value),secondary_y=False)

fig2.update_layout(
    xaxis_title="Year",
    legend_title="Age group & country",
    title="GDP vs Unemployment (" + country_w.value + " | " + age_group_w.value + ")",
)

fig2.update_yaxes(title_text="GDP in million euro", secondary_y=False)
fig2.update_yaxes(title_text="Unemployment percentage", secondary_y=True)


def update(change):
  with fig2.batch_update():

    unemployment_selection_i = unemployment[(unemployment["geo"] == country_w.value) & (unemployment["age"] == age_group_w.value)].index[0]
    fig2.data[0].y = unemployment.iloc[unemployment_selection_i,9:]
    fig2.data[0].name = age_group_w.value

    bip_selection_i = bip[bip["geo"] == country_w.value].index[0]
    fig2.data[1].y = bip.iloc[bip_selection_i,1:]
    fig2.data[1].name = country_w.value
    
    fig2.layout.title = go.layout.Title(text="GDP vs Unemployment, " + country_w.value + " | " + age_group_w.value)

    


country_w.observe(update)
age_group_w.observe(update)

display(*[country_w, age_group_w, fig2])

Dropdown(description='Country:', index=5, options=('AUT', 'BEL', 'BGR', 'CYP', 'CZE', 'DEU', 'DNK', 'EST', 'GR…

Dropdown(description='Agegroup:', options=('Y15-24', 'Y15-74', 'Y25-74'), value='Y15-24')

FigureWidget({
    'data': [{'name': 'Y15-24',
              'type': 'scatter',
              'uid': 'db6ad419…

In [190]:
unemployment_melted = unemployment[unemployment["age"]=="Y25-74"].melt(id_vars="geo", value_vars=unemployment.columns[9:], var_name="year")
unemployment_melted["geo"] = unemployment_melted["geo"].astype(str)
unemployment_melted["value"] = unemployment_melted["value"].astype(np.float32)
unemployment_melted["year"] = unemployment_melted["year"].astype(int)

In [256]:
fig4 = go.FigureWidget(make_subplots(rows=1, cols=2,
                                     specs = [[{'type': 'choropleth'} for c in np.arange(2)] for r in np.arange(1)],))

fig4.add_trace(
    go.Choropleth(
        locations=bip_melted["geo"],
        z=bip_melted['value'],
        zmin = bip_melted["value"].min(),
        zmax = bip_melted["value"].max(),
        colorbar=go.choropleth.ColorBar(x=-0.1,
                                        title=go.choropleth.colorbar.Title(text="GDP in million EUR"),
                                        titleside="right")
    ),
    row=1, col=1
)

fig4.add_trace(
    go.Choropleth(
        locations=unemployment_melted["geo"],
        z=unemployment_melted['value'],
        zmin = unemployment_melted["value"].min(),
        zmax = unemployment_melted["value"].max(),
        colorscale="Viridis",
        colorbar=go.choropleth.ColorBar(title=go.choropleth.colorbar.Title(text="Unemployment rate"),
                                        titleside="right")
    ),
    row=1, col=2
)

fig4.update_geos(resolution=50)
fig4.update_layout(margin={"r":60,"t":60,"l":50,"b":50})

fig4.update_layout(
    title_text = 'GDP and unemployment rate in Europe',
    **{'geo_scope' : "europe",'geo2_scope':"europe"}
    )


fig4.show()

In [298]:
unemployment_melted["type"]="Unemployment"

In [300]:
bip_melted["type"] = "GDP"

In [301]:
merged_melts = pd.concat([bip_melted,unemployment_melted])

In [328]:
merged_melts.drop(merged_melts[merged_melts["year"] == 2010].index, inplace=True)

In [380]:
fig5 = px.choropleth(
          merged_melts,
          locations='geo', 
          color='value',
          scope="europe",
          range_color=(bip_melted["value"].min(), bip_melted["value"].max()),
          height=600,
          title="GDP vs unemployment in Europe",
          animation_frame="year",
          basemap_visible=True,
          facet_col="type"
        )


fig5.update_geos(resolution=50)
fig5.update_layout(margin={"r":60,"t":60,"l":50,"b":10})

for i, t in enumerate(fig5.data):
    t.update(coloraxis=f"coloraxis{i+1}")
for fr in fig5.frames:
    for i, t in enumerate(fr.data):
        t.update(coloraxis=f"coloraxis{i+1}")

fig5.update_layout(
    coloraxis={
        "colorbar": {
            "x": -0.2, 
             "title":{"side":"right","text":"GDP in million EUR"}
             }
             },
    coloraxis2={
        "colorbar": {
            "x": 1.2,
            "title":{"side":"right","text":"Unemployment rate"},
        },
        "colorscale": "Viridis",
    }
)

fig5.show()